# Markov Chain

**Learning Objectives:** Learn how to build Markov Chains from n-grams and generate new sequences from the Markov Chains.

In [1]:
import types
from itertools import islice
import random

## n-grams

You can read about the background information related to n-grams [here](https://en.wikipedia.org/wiki/N-gram). Write a function, `build_ngrams`, that returns n-grams from an input sequene (iterator). Try to do this without concrete lists. The `islice` function may be helpful.

In [2]:
def build_ngrams(itr, n=2):
    for i in range(len(itr)-(n-1)):
        yield tuple(islice(itr, i, n + i))

In [3]:
b = build_ngrams(range(10), n=5)
list(b)

[(0, 1, 2, 3, 4),
 (1, 2, 3, 4, 5),
 (2, 3, 4, 5, 6),
 (3, 4, 5, 6, 7),
 (4, 5, 6, 7, 8),
 (5, 6, 7, 8, 9)]

In [4]:
[(i,i+1,i+2,i+3,i+4) for i in range(6)]

[(0, 1, 2, 3, 4),
 (1, 2, 3, 4, 5),
 (2, 3, 4, 5, 6),
 (3, 4, 5, 6, 7),
 (4, 5, 6, 7, 8),
 (5, 6, 7, 8, 9)]

In [5]:
a = build_ngrams(range(10), n=2)
assert hasattr(a, '__iter__') and not isinstance(a, list)
al = list(a)
assert al == [(i,i+1) for i in range(9)]

b = build_ngrams(range(10), n=5)
assert hasattr(b, '__iter__') and not isinstance(b, list)
bl = list(b)
assert bl == [(i,i+1,i+2,i+3,i+4) for i in range(6)]
assert list(build_ngrams('one two three four five six seven'.split(' '), n=5)) == \
    [('one','two','three','four','five'),
     ('two','three','four','five','six'),
     ('three','four','five','six','seven')]

## Markov chain

You can read about the background of Markov Chains [here](https://en.wikipedia.org/wiki/Markov_chain). Write a function `build_chain`, that returns a Markov Chain for a sequence of n-grams. This function should return a `dict` with:

* The keys will be the source node in the Markov Chain, which is the first `n-1` elements in each n-gram, as a `tuple`.
* The values will be a *list* of possible targets in the Markov Chain. As you find new values for a single key, you will need to append to the list.

In [6]:
def build_chain(ngrams, chain=None):
    if chain is None:
        chain = {}
    
    for ele in ngrams:
        n = len(ele)-1
        key = ele[:n:]
        value = ele[n::]
        val = min(value)
        if key in chain:
            chain[key].append(val)
        else:
            chain[key] = [val]
    return chain
    

In [7]:
random.seed(0)
seq1 = [random.randint(0,10) for i in range(200)]
chain = build_chain(build_ngrams(seq1, n=3))
seq2 = [random.randint(0,10) for i in range(200)]
chain = build_chain(build_ngrams(seq2, n=3), chain=chain)

In [8]:
random.seed(0)
seq1 = [random.randint(0,10) for i in range(200)]
chain = build_chain(build_ngrams(seq1, n=3))
seq2 = [random.randint(0,10) for i in range(200)]
chain = build_chain(build_ngrams(seq2, n=3), chain=chain)
assert chain[(0,0)]==[7, 10, 0, 3, 4]
assert chain[(4,2)]==[1, 3, 8, 3, 7, 1, 10, 2, 8]
assert len(chain.keys())==111

Write a function, `generate_sequence`, that can generate new sequences of length `m` from a trained Markov Chain (in the `dict` format described above). For the initial part of the sequence, randomly choose one of the keys in the Markov Chain `dict`.

In [9]:
import random

def generate_sequence(chain, m):
    random_key = random.choice(list(chain.keys()))
    tuple_size = len(random_key) 
    seq = list(random_key)
    
    for index in range(m - tuple_size):
        val = random.choice(chain[random_key])
        seq.append(val)
        random_key = tuple(seq[len(seq)-tuple_size::])
        
    return seq

In [10]:
random.seed(0)
seq3 = [random.randint(0,10) for i in range(200)]
chain2 = build_chain(build_ngrams(seq1, n=3))
generate_sequence(chain2, 10)

[8, 0, 1, 8, 10, 6, 8, 4, 8, 9]

In [11]:
random.seed(0)
seq3 = [random.randint(0,10) for i in range(200)]
chain2 = build_chain(build_ngrams(seq1, n=3))
generate_sequence(chain2, 10)

chain3 = build_chain(build_ngrams(seq1, n=5))
generate_sequence(chain3, 10)

[4, 1, 8, 5, 8, 3, 9, 8, 9, 4]

In [12]:
random.seed(0)
seq3 = [random.randint(0,10) for i in range(200)]
chain2 = build_chain(build_ngrams(seq1, n=3))
assert list(generate_sequence(chain2, 10))==[8, 0, 1, 8, 10, 6, 8, 4, 8, 9]
chain3 = build_chain(build_ngrams(seq1, n=5))
assert list(generate_sequence(chain3, 10))==[4, 1, 8, 5, 8, 3, 9, 8, 9, 4]

## Scrape the web to find lyrics

For this part of the exercise, you will need to find lyrics from one of your favorite bands, and use the [requests](http://docs.python-requests.org/en/latest/) and [BeautifulSoup](http://www.crummy.com/software/BeautifulSoup/bs4/doc/) packages to scrape the lyrics from a website. Some guidance:

1. The more songs the better (many dozens).
2. No hand downloading or editing of the files, you must do everything from code.
3. Save all of the lyrics in a single text file in this directory.

I provide an example here of doing this for all of U2's lyrics. You will have to modify this code significantly for the band of your choice.

In [13]:
import requests
from bs4 import BeautifulSoup

First get the page that has an index of all the lyrics and create a list of the URLs of those pages:

In [14]:
def get_lyric_urls():
    index = requests.get('http://www.metrolyrics.com/katy-perry-lyrics.html')
    soup = BeautifulSoup(index.text, 'html.parser')
    lyric_paths = [link.get('href') for link in
                   soup.find_all('div', class_='switchable lyrics clearfix')[0].find_all('a')]
    lyric_urls = [path for path in lyric_paths]
    lyric_urls = lyric_urls[1::]
    lyric_urls = lyric_urls[:len(lyric_urls) - 4:]
    return lyric_urls

In [15]:
lyric_urls = get_lyric_urls()

Here is a function that takes the URL of a single lyric page and scrapes the actual lyric as text:

In [16]:
def get_lyric(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    html_lyrics = soup.find_all('div', class_='js-lyric-text')[0].find_all('p')
    html_lyrics = [l.getText() for l in html_lyrics]
    return '\n'.join(html_lyrics)

This gets all of the lyrics. Note the `time.sleep(1.0)`. When scraping websites, it is often important to throttle your requests so as to not get banned from the website.

In [17]:
import time

def get_all_lyrics(lyric_urls):
    for url in lyric_urls:
        time.sleep(1.0)
        yield get_lyric(url)

In [18]:
lyrics = get_all_lyrics(lyric_urls)

Now save all the lyrics to a text file:

In [19]:
with open('all_katty_Perry_lyrics.txt', 'w') as f:
    for lyric in lyrics:
        f.write(lyric.replace('\r\n', '\n'))
        f.write('\n')

Leave the following cell to grade your code for this section:

In [20]:
assert True

## Generate new lyrics with the Markov chain

Here is the fun part!

In [21]:
import textwrap

Here are some simple function for tokenizing the lyrics:

In [22]:
import quicktoken

Read in your lyric file, tokenize the text (no stop words!) and generate new song lyrics. Some things to think about:

* This will work best if you generate new lines of text of some finite length (10-30 words).
* Use `textwrap.wrap` to format these lines and separate them using newlines.
* To get interesting results, you may need to run it multiple times.

In [24]:
get_chain = {}
with open('all_katty_Perry_lyrics.txt', 'r') as f:
    lyrics = f.read()

lyric_dic = {}
lyric_list = []
resize_lyrics = textwrap.wrap(lyrics)

for lyric in resize_lyrics:
    lyr = lyric.split(' ');
    for ele in lyr:
        lyric_list.append(ele)
        
index = 0
for lyric in lyric_list:
    if index not in lyric_dic:
        lyric_dic[index] = [lyric]
    elif len(lyric_dic[index]) < 10:
        lyric_dic[index].append(lyric)
    else:
        index += 1
        lyric_dic[index] = [lyric]


In [28]:
get_chain = {}
get_chain = quicktoken.files_to_lines(['all_katty_Perry_lyrics.txt'])
seq = quicktoken.tokenize_lines(get_chain)
get_chain = build_chain(build_ngrams(list(seq), n=3))

In [42]:
for i in range(50):
    [print(ele, ) for ele in generate_sequence(get_chain, 30)]

i'm
filling
my
cup
i
say
why
not
if
i
had
a
clue
never
planned
that
one
international
smile
please
fasten
your
seatbelts
and
make
a
treat.
cause
baby
don't
and
harder
than
ever
before
we
go
higher
and
higher
i
feel
the
fairytale
feeling
no
am
i
living
for?
at
the
super
rica
grabbing
tacos
checking
out
hotties
now
your
looking
for
a
perfect
storm
a
perfect
storm
a
perfect
storm
a
perfect
storm
a
perfect
storm
'cause
once
you're
mine
there's
no
need
to
fear
these
growing
pains
on
tea
on
the
end
of
the
blue
when]
crashing
from
the
trees
and
grew
tall
and
they
started
walking
and
the
artist
always
leaves
a
trail
of
stardust
'cause
she's
sweet
as
pie
but
if
you
wanna
stay
then
stay
'cause
i
am
enough
possible
to
be
a
do
you
know
this
maze
is
as
simple
as
it
seems
you
speak
to
me
even
in
my
jeep
snoop
doggy
dogg
on
the
beach
on
a
roller
coaster
can't
get
off
this
ride
you
change
your
mind
we
were
burning
i
tip
toe
through
your
triggered
mine
you
fight
me
but
now
she's
stuck
deep
in
cement
wi